In [10]:
import pandas as pd
import comet_ml


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from  decouple import Config,RepositoryEnv
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler


config = Config(RepositoryEnv(".env"))

In [11]:
def iniate_comet():
    experiment = comet_ml.Experiment(
  api_key= config("COMET_API"),
  project_name=config("PROJECT_NAME"),
  workspace=config("WORKSPACE")
)
    return experiment
    
experiment = iniate_comet()
    

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/dataspelunker/sparkify/f7f92b873a3f4e8dba0c7f213881271c
COMET INFO:   Uploads:
COMET INFO:     conda-environment-definition : 1
COMET INFO:     conda-info                   : 1
COMET INFO:     conda-specification          : 1
COMET INFO:     environment details          : 1
COMET INFO:     filename                     : 1
COMET INFO:     git metadata                 : 1
COMET INFO:     git-patch (uncompressed)     : 1 (356.57 KB)
COMET INFO:     installed packages           : 1
COMET INFO:     notebook                     

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/dataspelunker/sparkify/25e1f9358452494282650353da44cbfd



In [12]:

def normalize_dataframe(df, columns_to_normalize=None):
    if columns_to_normalize is None:
        columns_to_normalize = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

    scaler = MinMaxScaler()
    df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

    return df

def plot_conf_matrix(y_test,preds):
    cm = confusion_matrix(y_test,preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()

In [17]:

# Read the CSV file into a DataFrame
df = pd.read_csv("/Users/jacobfletcher/git/churn_project/data/lg_reduced_features.csv")

# Drop the 'Unnamed: 0' column
df = df.drop(['Unnamed: 0'], axis=1)
df = df.fillna(0)

# Split the DataFrame into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('label', axis=1), df['label'], train_size=0.8)

X_test = normalize_dataframe(X_test)
X_train = normalize_dataframe(X_train)

# Create a Random Forest Classifier
rfc = RandomForestClassifier(max_depth=2, random_state=42)

# Fit the model to the training data
rfc.fit(X_train, y_train)


RandomForestClassifier(max_depth=2, random_state=42)

In [18]:
preds = rfc.predict(X_test)
f1 = f1_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)


metrics = {"f1": f1, "recall": recall, "precision": precision}
print(metrics)

{'f1': 0.0, 'recall': 0.0, 'precision': 0.0}


/Users/jacobfletcher/anaconda3/envs/sparkify_churn/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
experiment.log_metrics(metrics)
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/dataspelunker/sparkify/25e1f9358452494282650353da44cbfd
COMET INFO:   Metrics:
COMET INFO:     f1        : 0.0
COMET INFO:     precision : 0.0
COMET INFO:     recall    : 0.0
COMET INFO:   Uploads:
COMET INFO:     conda-environment-definition : 1
COMET INFO:     conda-info                   : 1
COMET INFO:     conda-specification          : 1
COMET INFO:     environment details          : 1
COMET INFO:     filename                     : 1
COMET INFO:     git metadata                 : 1
COMET INFO:     git-patch (uncompressed)     : 1 (356.57 KB)
COMET INFO:     installed packages           : 1
COMET INFO:     notebook          